In [5]:
import json
import pandas as pd
from cafu.metadata import path
path_odds = path('dir_results')+'/odds/campeonato=franca_index=0.json'
r = open(path_odds, 'r')
odds = json.load(r)

In [23]:
pd.set_option('display.max_columns', None)
from cafu.utils import get_spark
spark = get_spark(1)

In [18]:
path_resumo = path('datalake')+'/partidas/resumo/brasil/df_resumo'
df = spark.read.parquet(path_resumo)

In [19]:
df.count()

436

In [20]:
df.dropna().count()

401

In [28]:
columns_null = [c for c in df.columns if df.filter(f'{c} IS NULL').count()>0]

In [29]:
columns_null

['formacao_time_casa',
 'formacao_time_visitante',
 'time_casa_defesas',
 'time_casa_faltas_cometidas',
 'time_visitante_defesas',
 'time_visitante_faltas_cometidas']

In [30]:
c = columns_null[0]
df.filter(f'{c} IS NULL').toPandas()

,campeonato,campeonato_metadata,temporada_metadata,date,formacao_time_casa,formacao_time_visitante,jogo_id,time_casa,time_casa_cartoes_amarelos,time_casa_cartoes_vermelhos,time_casa_chutes_fora,time_casa_chutes_nogol,time_casa_defesas,time_casa_escanteios,time_casa_faltas_cometidas,time_casa_gols_feitos,time_casa_impedimentos,time_casa_posse,time_visitante,time_visitante_cartoes_amarelos,time_visitante_cartoes_vermelhos,time_visitante_chutes_fora,time_visitante_chutes_nogol,time_visitante_defesas,time_visitante_escanteios,time_visitante_faltas_cometidas,time_visitante_gols_feitos,time_visitante_impedimentos,time_visitante_posse,date_update
0,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-03,None,None,569651,Red Bull Bragantino,3,0,11,10,0,15,14,2,2,0.52,Atlético-GO,3,0,7,1,0,7,6,0,1,0.48,2022-03-16 12:02:16.299781
1,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-07,None,None,569643,Red Bull Bragantino,4,0,8,4,0,7,19,1,1,0.45,Flamengo,2,0,10,6,0,10,15,1,4,0.54,2022-03-16 11:59:13.340158
2,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-14,None,None,569638,Vasco da Gama,1,0,7,0,0,14,9,0,2,0.64,Internacional,6,0,9,7,0,6,20,2,0,0.35,2022-03-16 11:57:22.118010
3,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-21,None,None,569624,Goiás,2,0,13,1,0,3,18,0,1,0.43,Red Bull Bragantino,0,0,9,5,0,21,20,0,1,0.56,2022-03-16 11:52:08.474721
4,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-14,None,None,569630,Athletico-PR,3,0,5,3,0,16,14,2,4,0.59,Atlético-GO,1,0,7,4,0,2,9,1,3,0.40,2022-03-16 11:54:22.913357
5,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-15,None,None,569633,Sport,3,0,4,0,0,2,9,0,0,0.31,Red Bull Bragantino,0,0,16,6,0,24,10,0,1,0.69,2022-03-16 11:55:30.199970
6,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-21,None,None,569629,Corinthians,1,0,10,4,0,11,21,0,1,0.67,Vasco da Gama,2,0,7,0,0,9,14,0,5,0.32,2022-03-16 11:54:01.878312
7,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-10,None,None,569641,Corinthians,3,0,9,8,0,2,17,3,0,0.44,Athletico-PR,0,0,6,6,0,6,11,3,0,0.55,2022-03-16 11:58:28.369566
8,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-10,None,None,569649,Fortaleza,2,0,11,6,0,3,15,3,1,0.43,Vasco da Gama,3,0,11,1,0,17,13,0,1,0.56,2022-03-16 12:01:30.939767
9,2020 Brasileiro Serie A,brasil,2020-2020,2021-02-22,None,None,569621,Palmeiras,2,0,15,4,0,16,20,1,0,0.63,Atlético-GO,1,0,6,3,0,7,13,1,0,0.36,2022-03-16 11:50:59.623446


In [8]:
dfs = {}
for t in odds:
    try:
        dfs[t] = pd.DataFrame([odds[t]])
    except:
        print(t)

In [10]:
for t in odds:
    print(t)

Vitória / Empate / Vitória, Tempo Regular
Vitória / Empate / Vitória, Primeiro Tempo
Vitória / Empate / Vitória, Segundo Tempo
Handicap Asiático, Tempo Regular
Handicap Asiático, Primeiro Tempo
Mais de / Menos de - Saint-Étienne, Tempo Regular
Mais de / Menos de - Lille, Tempo Regular
Mais de / Menos de, Tempo Regular
Ambos os tempos acima de 1.5 gols?, Tempo Regular
Ambos os tempos abaixo de 1.5 gols?, Tempo Regular
Mais de / Menos de - Saint-Étienne, Primeiro Tempo
Mais de / Menos de - Lille, Primeiro Tempo
Mais de / Menos de, Primeiro Tempo
Mais de / Menos de - Saint-Étienne, Segundo Tempo
Mais de / Menos de - Lille, Segundo Tempo
Mais de / Menos de, Segundo Tempo
Time que marca o 1º Gol, Tempo Regular
Time que marca o 1º Gol, Primeiro Tempo
Qual time a marcar (sem receber gol), Tempo Regular
Handicap Europeu, Tempo Regular
1X2 Handicap Fixo, Primeiro Tempo
Dupla Chance, Tempo Regular
Dupla Chance, Primeiro Tempo
Empate Anula, Tempo Regular
Empate Anula, Primeiro Tempo
Quem marca ma

In [15]:
dfs['Exact Corners - Saint-Étienne, Primeiro Tempo']

,0-1,2,3,4+
0,1.79,3.75,6.20,7.40
